In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from biopsykit.utils.dataframe_handling import multi_xs

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fau_colors import register_cmaps
register_cmaps()

sns.set_theme(context="talk", style="white", palette="faculties", font_scale=1.2)

%matplotlib widget

In [ ]:
df = pd.read_pickle("full_df.pkl")

In [ ]:
df = df.rename(index={"specifity":"specificity"})
df

# 5 stage

In [ ]:
df_5 = multi_xs(df, ["5stage"], level="stage")
df_5_mcc = multi_xs(df_5, ["mcc"],level="metric")[["data"]].astype(float)
df_5 = multi_xs(df_5, ["accuracy", "f1", "precision", "recall", "specificity"],level="metric")[["data"]].astype(float)

In [ ]:
df_5_mean = df_5.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
#df_5_mean = df_5_mean.unstack("modality")["data"]["mean"][["ACT", "ACT + HRV", "ACT + HRV + RRV", "ACT + HRV + EDR"]]
df_5_mean = df_5_mean.unstack("metric")["data"].round(decimals=1).applymap('{:,.1f}'.format).applymap('{:0>4}'.format)

In [ ]:
df_5_mcc = df_5_mcc.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
df_5_mcc = df_5_mcc.unstack("metric")["data"].round(decimals=2).applymap('{:,.2f}'.format).applymap('{:0>4}'.format)

In [ ]:
#df_5_mean.sort_values(by=["accuracy"], axis=0, ascending=False).sort_index(level="modality", sort_remaining=False)

In [ ]:
df_5_mean = pd.DataFrame(
    "$" + df_5_mean["median"].astype(str) + " (" + df_5_mean["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_5_mean = df_5_mean.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [ ]:
df_5_mcc = pd.DataFrame(
    "$" + df_5_mcc["median"].astype(str) + " (" + df_5_mcc["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_5_mcc = df_5_mcc.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [ ]:
df_5_mean = df_5_mean.join(df_5_mcc).reindex(["accuracy", "f1", "mcc", "precision", "recall", "specificity"], axis=1)

In [ ]:
df_5_mean.style.to_latex("5stage.tex")

In [ ]:
df_5_mean

# 3 stage

In [ ]:
df_3 = multi_xs(df, ["3stage"], level="stage")
df_3_mcc = multi_xs(df_3, ["mcc"],level="metric")[["data"]].astype(float)
df_3 = multi_xs(df_3, ["accuracy", "f1", "precision", "recall", "specificity"],level="metric")[["data"]].astype(float)

In [ ]:
df_3_mean = df_3.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
#df_3_mean = df_3_mean.unstack("modality")["data"]["mean"][["ACT", "ACT + HRV", "ACT + HRV + RRV", "ACT + HRV + EDR"]]
df_3_mean = df_3_mean.unstack("metric")["data"].round(decimals=1).applymap('{:,.1f}'.format).applymap('{:0>4}'.format)

In [ ]:
df_3_mcc = df_3_mcc.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
df_3_mcc = df_3_mcc.unstack("metric")["data"].round(decimals=2).applymap('{:,.2f}'.format).applymap('{:0>4}'.format)

In [ ]:
#df_3_mean.sort_values(by=["accuracy"], axis=0, ascending=False).sort_index(level="modality", sort_remaining=False)

In [ ]:
df_3_mean = pd.DataFrame(
    "$" + df_3_mean["median"].astype(str) + " (" + df_3_mean["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_3_mean = df_3_mean.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [ ]:
df_3_mcc = pd.DataFrame(
    "$" + df_3_mcc["median"].astype(str) + " (" + df_3_mcc["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_3_mcc = df_3_mcc.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [ ]:
df_3_mean = df_3_mean.join(df_3_mcc).reindex(["accuracy", "f1", "mcc", "precision", "recall", "specificity"], axis=1)

In [ ]:
df_3_mean.style.to_latex("3stage.tex")

In [ ]:
df_3_mean

# 2 stage

In [ ]:
df_2 = multi_xs(df, ["Binary"], level="stage")
df_2_mcc = multi_xs(df_2, ["mcc"],level="metric")[["data"]].astype(float)
df_2 = multi_xs(df_2, ["accuracy", "f1", "precision", "recall", "specificity"],level="metric")[["data"]].astype(float)

In [ ]:
df_2_mean = df_2.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
#df_2_mean = df_2_mean.unstack("modality")["data"]["mean"][["ACT", "ACT + HRV", "ACT + HRV + RRV", "ACT + HRV + EDR"]]
df_2_mean = df_2_mean.unstack("metric")["data"].round(decimals=1).applymap('{:,.1f}'.format).applymap('{:0>4}'.format)

In [ ]:
df_2_mcc = df_2_mcc.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
df_2_mcc = df_2_mcc.unstack("metric")["data"].round(decimals=2).applymap('{:,.2f}'.format).applymap('{:0>4}'.format)

In [ ]:
#df_2_mean.sort_values(by=["accuracy"], axis=0, ascending=False).sort_index(level="modality", sort_remaining=False)

In [ ]:
df_2_mean = pd.DataFrame(
    "$" + df_2_mean["median"].astype(str) + " (" + df_2_mean["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_2_mean = df_2_mean.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [ ]:
df_2_mcc = pd.DataFrame(
    "$" + df_2_mcc["median"].astype(str) + " (" + df_2_mcc["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_2_mcc = df_2_mcc.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [ ]:
df_2_mean = df_2_mean.join(df_2_mcc).reindex(["accuracy", "f1", "mcc", "precision", "recall", "specificity"], axis=1)

In [ ]:
df_2_mean.style.to_latex("2stage.tex")

In [ ]:
df_2_mean.style.to_latex("2stage.tex")

In [ ]:
df_2_mean